In [ ]:
!gdown --id '153DyBFeQCCl3-85M0SalJz8z0W_ZKYX5' --output parking.zip # 下载dataset
!unzip parking.zip

In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

def readfile(path, label):
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.full((len(image_dir)), label, dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        x[i, :, :] = cv2.resize(img, (128, 128))
    return x, y

In [3]:
workspace_dir = './parking'
print("Reading training data occupy")
train_occ_x, train_occ_y = readfile(workspace_dir + '/train/occupy', 1)
print("Reading training data free")
train_free_x, train_free_y = readfile(workspace_dir + '/train/free', 0)
print('generating training data')
train_x = np.append(train_occ_x, train_free_x, axis=0)
train_y = np.append(train_occ_y, train_free_y, axis=0)
del train_occ_x, train_occ_y, train_free_x, train_free_y

print("Reading testing data occupy")
test_occ_x, test_occ_y = readfile(workspace_dir + '/test/occupy', 1)
print("Reading testing data free")
test_free_x, test_free_y = readfile(workspace_dir + '/test/free', 0)
print('generating testing data')
test_x = np.append(test_occ_x, test_free_x, axis=0)
test_y = np.append(test_occ_y, test_free_y, axis=0)
del test_occ_x, test_occ_y, test_free_x, test_free_y

Reading training data occupy
Reading training data free
generating training data
Reading testing data occupy
Reading testing data free
generating testing data


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Convolution2D(25, 3, 3, input_shape=train_x[0].shape),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Convolution2D(50, 3, 3),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.fit(train_x, train_y, epochs=30)

In [5]:
model.evaluate(test_x, test_y, verbose=2)

23/23 - 0s - loss: 0.0172 - accuracy: 0.9932


[0.017180217429995537, 0.9932065010070801]